## Sucesses and Failures of Hartree-Fock


In this example, we will quantify the different sources for error of the Hartree Fock method on the example of the H$_2$ molecule.

In [ ]:
from ase.build import molecule
from ase.atoms import Atoms
import numpy as np
import matplotlib.pyplot as plt
import psi4
from psi4_helper import geom_ase_to_psi4

diatomic = 'H2'

In [ ]:
print('.', end='')
atoms = Atoms(diatomic)
atoms.positions[1,2] = 1.5
geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
  #initiate Psi4 molecule object
h2 = psi4.geometry(geom_input)

  # Set computation options
psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'pk',
                  #'frozen_docc': [0],
                  #'active':[4],
                  #'frozen_uocc': [0],
                  'e_convergence': 1e-8})
E,wfn = psi4.energy('scf',return_wfn=True)
psi4.molden(wfn,'c2.molden')

In [ ]:
psi4.core.clean()
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('6000 MB')

distances = [0.4,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.4,1.8,2.2,3.0,3.5,4.5,6,10.]

E_HF = []
E_exact = []

for i in distances:
  print('. ', end='')
  atoms = Atoms(diatomic)
  atoms.positions[1,2] = i
  geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
  #initiate Psi4 molecule object
  h2 = psi4.geometry(geom_input)

  # Set computation options
  psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'direct',
                  'frozen_docc': [0],
                  'active':[2],
                  'frozen_uocc': [0],
                  'ci_maxiter': 100,
                  'e_convergence': 1e-8})

  E_HF.append(psi4.energy('scf',return_wfn=False))
  E_exact.append(psi4.energy('fci',return_wfn=False))
  print (i, E_HF[-1], E_exact[-1])

print('calculations finished!')
#for x, i,j in zip(distances,E_HF, E_exact):
#    print(x, i,j)

In [ ]:
%matplotlib widget
f1 = plt.figure()
plt.plot(distances, E_HF,'b',label='HF')
plt.plot(distances, E_exact,'r',label='exact')
plt.legend()
plt.xlabel(r'H-H distance in Angstrom $\AA$')
plt.ylabel(r'energy in Hartree [$E_H$]')
plt.show()



The exact solution includes all possible Slater determinants and correctly describes bond breaking. This is, however, in almost all cases impossible to calculate.

We can find a solution to the correlation problem in two steps:
* first we account for static correlation in the bond breaking limit. This will treat all H-H distances equally
* Then we can include dynamic correlation

In [ ]:
E_CASSCF = []

psi4.core.set_output_file('output.dat', True)
distances = [0.4,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.4,1.8,2.2,3.0,3.5,4.5,6,10.]

for i in distances:
  psi4.core.clean()
  print('. ', end='')
  atoms = Atoms(diatomic)
  atoms.positions[1,2] = i
  geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
  #initiate Psi4 molecule object
  h2 = psi4.geometry(geom_input)

  # Set computation options
  psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type': 'pk',
                  'mcscf_algorithm':'ah',
                  'qc_module': 'detci',
                  'ci_maxiter': 100,    
                    'mcscf_maxiter': 300,
                  'e_convergence': 1.e-8,
                  'num_roots':1,
                   })

  #cisd_energy, cisd_wfn = psi4.energy("cisd", return_wfn=True)
  #4 orbitals [sigma, sigma*, and two more), The first 2 of them should be active
  #That means that CASSCF should build the possible determinants from those two
  psi4.set_options({
      'restricted_docc': [0],
      'active': [2],
      'restricted_uocc':[2],
                   })
    
  E = psi4.energy('casscf')#,ref_wfn=cisd_wfn)
  print(i, E)
  E_CASSCF.append(E) 

print('calculations finished!')
#for x, i in zip(distances,E_CASSCF):
#    print(x, i)


In [ ]:
%matplotlib widget
f2 = plt.figure()
plt.plot(distances, E_HF,'b',label='HF')
plt.plot(distances, E_CASSCF, 'g', label='CASSCF')
plt.plot(distances, E_exact,'r',label='exact')
plt.legend()
plt.xlabel(r'H-H distance in Angstrom $\AA$')
plt.ylabel(r'energy in Hartree [$E_H$]')
plt.show()